In [22]:
import geopandas
polys = geopandas.read_file("PRAHA_P.json")

polys



,OBJECTID,NAZEV,ID_POSKYT,KOD_OBEC,DAT_VZNIK,STAV_ZMENA,DAT_ZMENA,Shape_Length,Shape_Area,geometry
0,1,Praha hl. m.,43,1,20170613,None,None,1.912362,0.062316,"POLYGON ((14.22444 50.10297, 14.22444 50.10300..."


In [23]:
grid = geopandas.read_file("grid_point.json")

grid

,distance,latitude,longitude,geometry
0,14.558532,49.9418,14.2244,POINT (14.22440 49.94180)
1,13.914935,49.9418,14.2344,POINT (14.23440 49.94180)
2,13.332824,49.9418,14.2444,POINT (14.24440 49.94180)
3,12.553345,49.9418,14.2544,POINT (14.25440 49.94180)
4,11.532095,49.9418,14.2644,POINT (14.26440 49.94180)
...,...,...,...,...
1171,7.887784,50.1718,14.6644,POINT (14.66440 50.17180)
1172,8.086243,50.1718,14.6744,POINT (14.67440 50.17180)
1173,8.190965,50.1718,14.6844,POINT (14.68440 50.17180)
1174,8.441605,50.1718,14.6944,POINT (14.69440 50.17180)


In [31]:
mask = grid.within(polys.loc[0, 'geometry'])

grid['Praha'] = mask

novy_grid = grid[grid['Praha'] == True]

novy_grid

,distance,latitude,longitude,geometry,Praha
61,3.961135,49.9518,14.3444,POINT (14.34440 49.95180),True
62,3.642957,49.9518,14.3544,POINT (14.35440 49.95180),True
63,3.001230,49.9518,14.3644,POINT (14.36440 49.95180),True
64,2.220159,49.9518,14.3744,POINT (14.37440 49.95180),True
65,1.820723,49.9518,14.3844,POINT (14.38440 49.95180),True
...,...,...,...,...,...
1111,3.395039,50.1618,14.5544,POINT (14.55440 50.16180),True
1153,2.839633,50.1718,14.4844,POINT (14.48440 50.17180),True
1154,2.427423,50.1718,14.4944,POINT (14.49440 50.17180),True
1156,2.434255,50.1718,14.5144,POINT (14.51440 50.17180),True


In [21]:
novy_grid.to_file('Novy_grid.json', driver="GeoJSON")

In [49]:
import folium

moje_mapa = folium.Map(location=(50.07653, 14.40232), zoom_start=10, tiles="openstreetmap")


#tady si postupně vykreslím každý bod na mapu jako puntik, barvy jsou odstupnované podle nejmenší vzdálenosti
for index, row in novy_grid.iterrows():
  if row['distance'] < 1:
    color = "red"
  elif row['distance'] < 3:
    color = "green"
  else:
    color = "blue"
  # radius = 500
  dx = 0.005
  folium.Rectangle(
      bounds=[[row['latitude'] + dx, row['longitude'] + dx], [row['latitude'] - dx, row['longitude'] - dx]],
      # radius=radius,
      color=color,
      weight=1,
      fill_opacity=0.2,
      opacity=1,
      fill_color=color,
      fill=False,  # gets overridden by fill_color
      popup=row['distance'],
      tooltip="km",
  ).add_to(moje_mapa)

# folium.GeoJson("OPK_PrirodniParky_p.json").add_to(moje_mapa)

# folium.GeoJson("TMMESTSKECASTI_P.json").add_to(moje_mapa)

folium.GeoJson("PRAHA_P.json").add_to(moje_mapa)



# #tady si vykreslím náš dataset
# for c in interest.coordinates:
#   folium.Marker(
#       location=[c[1], c[0]],
#       tooltip="Opravdova cukrarna",
#       popup="Zatim bez popisku",
#       icon=folium.Icon(icon="star"),
#   ).add_to(moje_mapa)

# #zadní souřadnic okraje- může být i polygon
#   trail_coordinates = [
#     (NORTH, EAST),
#     (NORTH, WEST),
#     (SOUTH, WEST),
#     (SOUTH, EAST),
#     (NORTH, EAST),
#   ]

# #tady si vykreslím hranice, dá se využít polygon
# folium.PolyLine(trail_coordinates, tooltip="oblast nahody").add_to(moje_mapa)

# #uložení mapy jako internetové stránky, můžeme otevřít v prohlížeči
moje_mapa
